In [ ]:


#to show data cleaning, join tools, data search and requare order from below

#tasks in this script code
#the structure person_id, city, type_of_broadband_connection, name_of_isp, average_download_speed, average_upload_speed
#has 1 line per person (i.e. calculate a single average download and upload speed for each person)
#only contains people in the cities 'Samsville' and 'Databury'
#only contains download and upload measurements which have run successfully (i.e. put a filter on did_test_complete_successfully)
#only contains tests from the month of January 2021 (i.e. put a filter on time_of_measurement).

In [2]:

#data import
Dspeed_measure <- read.csv("download_speed_measurements.csv")
Uspeed_measure <- read.csv("upload_speed_measurements.csv")
person_details <- read.csv("details_for_each_person.csv")


In [3]:

#some measured speed data was 0 I assume that it means some disconnectivity and the network does not work in this time and I did not delete these data

library(data.table)
library(dplyr)
#only contains download and upload measurements which have run successfully (i.e. put a filter on did_test_complete_successfully)
Dspeed_measure <- data.table(na.omit(Dspeed_measure))
Uspeed_measure <- data.table(na.omit(Uspeed_measure))


Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [4]:
# we only need year & month & day

library(data.table)
replace <- as.character(Dspeed_measure$time_of_measurement)
replace <- sub("T.*", "", replace)
Dspeed_measure[,"time_of_measurement"] <- replace
Dspeed_measure <- setDT(Dspeed_measure)[time_of_measurement %between% c('2021-01-01', '2021-01-31')]

In [10]:
head(Dspeed_measure)

person_id,time_of_measurement,measured_download_speed_in_Mbps,did_test_complete_successfully
50868,2021-01-01,10.635198,TRUE
50868,2021-01-19,0.000000,TRUE
50868,2021-01-02,0.000000,TRUE
50868,2021-01-02,0.000000,TRUE
50868,2021-01-06,15.634309,TRUE
50868,2021-01-17,6.594722,TRUE


In [5]:

replace <- as.character(Uspeed_measure$time_of_measurement)
replace <- sub("T.*", "", replace)
Uspeed_measure[,"time_of_measurement"] <- replace
Uspeed_measure <- setDT(Uspeed_measure)[time_of_measurement %between% c('2021-01-01', '2021-01-31')]

In [11]:
head(Uspeed_measure)

person_id,time_of_measurement,measured_upload_speed_in_Mbps,did_test_complete_successfully
50868,2021-01-06,0.7703398,TRUE
50868,2021-01-18,0.4549614,TRUE
50868,2021-01-21,0.8439032,TRUE
50868,2021-01-23,0.8108615,TRUE
50868,2021-01-25,0.9719853,TRUE
50868,2021-01-28,0.0000000,TRUE


In [12]:

#has 1 line per person (i.e. calculate a single average download and upload speed for each person)
Dspeed_measure <- Dspeed_measure[,mean(measured_download_speed_in_Mbps), by = person_id]
Uspeed_measure <- Uspeed_measure[,mean(measured_upload_speed_in_Mbps), by = person_id]

In [13]:

#joining tables

New_person_detail <- merge(x = person_details, y = Dspeed_measure, by = "person_id", all.x = TRUE)
colnames(New_person_detail)[5] <- "average_download_speed"

New_person_detail <- merge(x = New_person_detail, y = Uspeed_measure, by = "person_id", all.x = TRUE)
colnames(New_person_detail)[6] <- "average_upload_speed"

In [14]:
#only contains people in the cities 'Samsville' and 'Databury'
New_person_detail <- New_person_detail[New_person_detail$city %in% c("Databury","Samsville"),]

In [16]:
head(New_person_detail)

person_id,city,type_of_broadband_connection,name_of_isp,average_download_speed,average_upload_speed
11218,Databury,VDSL,Fibrelicious,19.420619,5.1446965
11560,Samsville,ADSL,Useus,194.959863,78.1698964
11677,Samsville,ADSL,Fibrelicious,8.955024,0.5259603
12309,Samsville,Fibre,Useus,8.962087,0.5509977
12312,Databury,VDSL,Useus,17.177268,4.4410936
13623,Samsville,ADSL,Fibrelicious,8.884879,0.5471097
